## Ejercicios.

Los datos provistos son datos ficticios que representan los pagos que realiza una compañía de seguros para cada uno de los siniestros reportados durante el mes de Enero del Año 2023.

1. Cargue los datos que se encuentran en la carpeta `data`, tanto el archivo de `claims.csv`, como los archivos de `claims_payments.csv`.
2. Remueva los duplicados, y conserve unicamente un único registro por `claim_id`, y este, deberá ser el registro más nuevo de la base de datos según `claim_date`, y resuelva los siguientes puntos en base a este nuevo dataset.
3. Una vez realice este proceso, deberá responder las siguientes preguntas:
   1. Cuantos siniestros distintos han ocurrido por cada uno de los días?
   2. Existen días que no tienen ningún siniestro? Cuales son estos?
   3. Cuantos registros tienen **nulos** en su campo `claim_date`?. Que porcentaje del total de la base representan?
   4. Cuál es el día que tiene **más**, y el día que tiene **menos** siniestros reportados? Que porcentaje de del total de la base representa?
   5. Cuál es la marca de vehiculos que **más** presenta siniestros, y cual es la marca que **menos** siniestros presenta?
   6. Cuál es el valor de pago más alto que ha realizado la compañia?
   7. Muestre por marca de vehículo, los valores de pagos realizados por concepto de `PÉRDIDA PARCIAL`.
   8. Cuales son los **3 primeros** tipos de pagos **más** frecuente en la compañía?
   9. Cuál es el número de días **mínimo, máximo**, y **promedio**, que han transcurrido entre un siniestro y el pago de la compañia?
   10. Cuantos siniestros aún no han recibido ningún pago de la compañia? Que porcentaje de la base representan?


4. Exporte el archivo consolidado, el cual contiene para cada `claim_id` la información de los pagos realizados por la compañia. Cada archivo se debe guardar por día, usando la siguiente nomenclatura: `payment_report_yyyy_mm_dd.csv`, donde `yyyy` hace referencia al año del siniestro, `mm` representa el mes del siniestro y `dd` representa el día del siniestro. Ejemplo: `payment_report_2023_01_05.csv`, `payment_report_2023_01_06.csv`, `...`.
5. Mueva los archivos que exportó a la carpeta `reporting`, usando Python.

## Solución

In [20]:

import pandas as pd
import numpy as np
import os

In [7]:

claims_data = pd.read_csv('data/claims.csv', delimiter=';')
payments_data = pd.read_csv('data/payments.csv', delimiter=';', encoding='ISO-8859-1')

claims_head = claims_data.tail()
payments_head = payments_data.head()

claims_data['claim_date'] = pd.to_datetime(claims_data['claim_date'])
payments_data['payment_date'] = pd.to_datetime(payments_data['payment_date'])

claims_data.sort_values(by=['claim_id', 'claim_date'], ascending=[True, False], inplace=True)

claims_data_deduplicated = claims_data.drop_duplicates(subset='claim_id', keep='first')

claims_data_deduplicated.to_csv('reporting/claims_deduplicated.csv', index=False, sep=';')

print(claims_data_deduplicated.info())
print(claims_data_deduplicated.head())

<class 'pandas.core.frame.DataFrame'>
Index: 946 entries, 776 to 954
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   claim_id              946 non-null    int64              
 1   claim_date            911 non-null    datetime64[ns, UTC]
 2   vehicle_model         946 non-null    int64              
 3   vehicle_brand         946 non-null    object             
 4   vehicle_market_value  946 non-null    float64            
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(1)
memory usage: 44.3+ KB
None
     claim_id                claim_date  vehicle_model vehicle_brand  \
776        26 2023-01-03 00:00:00+00:00           2000     CHEVROLET   
12         36 2023-01-27 00:00:00+00:00           2012       PEUGEOT   
882        45 2023-01-06 00:00:00+00:00           2010     CHEVROLET   
230        46 2023-01-06 00:00:00+00:00           2019        NISSAN   
163   

In [8]:
siniestros_por_dia = claims_data_deduplicated.groupby(claims_data_deduplicated['claim_date'].dt.date)['claim_id'].count()
print(siniestros_por_dia)

claim_date
2023-01-02    30
2023-01-03    38
2023-01-04    31
2023-01-05    26
2023-01-06    27
2023-01-07    29
2023-01-08    35
2023-01-09    29
2023-01-10    31
2023-01-11    32
2023-01-12    27
2023-01-13    27
2023-01-14    33
2023-01-16    32
2023-01-17    29
2023-01-18    33
2023-01-19    29
2023-01-20    29
2023-01-21    37
2023-01-22    37
2023-01-23    47
2023-01-24    28
2023-01-25    26
2023-01-26    32
2023-01-27    23
2023-01-28    42
2023-01-29    19
2023-01-30    35
2023-01-31    38
Name: claim_id, dtype: int64


In [9]:
dias_sin_siniestros = siniestros_por_dia[siniestros_por_dia == 0]
print(dias_sin_siniestros)
if dias_sin_siniestros.empty:
    print("No hay días sin siniestros en los datos.")
else:
    print("Días sin siniestros:")
    print(dias_sin_siniestros)

Series([], Name: claim_id, dtype: int64)
No hay días sin siniestros en los datos.


In [10]:
registros_nulos = claims_data_deduplicated['claim_date'].isnull().sum()
porcentaje_nulos = (registros_nulos / len(claims_data_deduplicated)) * 100
print(f"Registros nulos en 'claim_date': {registros_nulos}, Porcentaje: {porcentaje_nulos:.2f}%")

Registros nulos en 'claim_date': 35, Porcentaje: 3.70%


In [11]:
dia_max_siniestros = siniestros_por_dia.idxmax()
dia_min_siniestros = siniestros_por_dia.idxmin()
max_siniestros = siniestros_por_dia.max()
min_siniestros = siniestros_por_dia.min()
porcentaje_max = (max_siniestros / len(claims_data_deduplicated)) * 100
porcentaje_min = (min_siniestros / len(claims_data_deduplicated)) * 100

print(f"Día con más siniestros ({max_siniestros} siniestros): {dia_max_siniestros}, Porcentaje: {porcentaje_max:.2f}%")
print(f"Día con menos siniestros ({min_siniestros} siniestros): {dia_min_siniestros}, Porcentaje: {porcentaje_min:.2f}%")

Día con más siniestros (47 siniestros): 2023-01-23, Porcentaje: 4.97%
Día con menos siniestros (19 siniestros): 2023-01-29, Porcentaje: 2.01%


In [12]:
marca_mas_siniestros = claims_data_deduplicated['vehicle_brand'].value_counts().idxmax()
marca_menos_siniestros = claims_data_deduplicated['vehicle_brand'].value_counts().idxmin()

print(f"Marca con más siniestros: {marca_mas_siniestros}")
print(f"Marca con menos siniestros: {marca_menos_siniestros}")

Marca con más siniestros: CHEVROLET
Marca con menos siniestros: MAZDA


In [13]:
valor_pago_maximo = payments_data['payment_value'].max()
print(f"Valor de pago más alto realizado por la compañía: {valor_pago_maximo}")

Valor de pago más alto realizado por la compañía: 99000000.0


In [14]:
datos_combinados = payments_data.merge(claims_data_deduplicated, left_on='claim_id', right_index=True, how='inner')

pagos_perdida_parcial = datos_combinados[datos_combinados['payment_type'] == 'PÉRDIDA PARCIAL']

pagos_por_marca = pagos_perdida_parcial.groupby('vehicle_brand')['payment_value'].sum()
print(pagos_por_marca)

vehicle_brand
CHEVROLET    517000000.0
MAZDA        140000000.0
NISSAN       231000000.0
PEUGEOT        8000000.0
RENAULT      182000000.0
SUBARU        99000000.0
TOYOTA       330000000.0
Name: payment_value, dtype: float64


In [ ]:
tipos_pagos_mas_frecuentes = payments_data['payment_type'].value_counts().head(3)
print(tipos_pagos_mas_frecuentes)

In [ ]:
diferencia_dias = (payments_data['payment_date'] - claims_data_deduplicated['claim_date']).dt.days.abs()

min_dias = diferencia_dias.min()
max_dias = diferencia_dias.max()
promedio_dias = diferencia_dias.mean()

print(f"Número mínimo de días: {min_dias}")
print(f"Número máximo de días: {max_dias}")
print(f"Número promedio de días: {promedio_dias:.2f}")

In [ ]:
siniestros_sin_pago = len(claims_data_deduplicated[~claims_data_deduplicated['claim_id'].isin(payments_data['claim_id'])])
porcentaje_sin_pago = (siniestros_sin_pago / len(claims_data_deduplicated)) * 100

print(f"Siniestros sin ningún pago: {siniestros_sin_pago}, Porcentaje: {porcentaje_sin_pago:.2f}%")

In [27]:

grouped_payments = payments_data.groupby(payments_data['payment_date'].dt.date)
for date, group in grouped_payments:

    file_name = f"payment_report_{date:%Y_%m_%d}.csv"
    
    file_path = file_name
    group.to_csv(file_path, index=False, sep=';')

    source_path = os.path.abspath(file_path)
    target_directory = 'reporting'
    target_path = os.path.abspath(f"{target_directory}/{file_name}")
    os.rename(source_path, target_path)